In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv', index_col='date_time')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv', index_col='date_time')


In [ ]:
train.head()


In [ ]:
test.head()

In [ ]:
print('Train Describe \n\n\n',train.describe(),'\n\n\n Test Describe\n\n\n',test.describe())

In [ ]:
train.isnull().sum() # To check the missing values

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
# Since there are no missing values we are good to go for EDA. We start with imports
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
sns.pairplot(train)

In [ ]:
X = train.iloc[:,:-3]
y = train.iloc[:,-3:]
X.head()

In [ ]:
y.head()

In [ ]:
# We shall use catboost for regression. 
from catboost import Pool, CatBoostRegressor

In [ ]:
train_pool = Pool(X, y)
test_pool = Pool(test) 

# specify the training parameters 
model = CatBoostRegressor(loss_function='MultiRMSE', verbose=False, iterations= 2000)
#train the model
model.fit(train_pool)
# make the prediction using the resulting model
preds = model.predict(test_pool)
#print(preds)

I used MultiRMSE as suggested in the following link,
https://github.com/catboost/catboost/issues/1417

The reason to do so is as mentioned in the link. Now we shall try to find out the performance. Since there were no labels in test set we shall devide the train set and check the performance.


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Now we will repeat the same modelling step
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test) 

# specify the training parameters 
model = CatBoostRegressor(loss_function='MultiRMSE', verbose=False, iterations= 10000)
#train the model
model.fit(train_pool)
# make the prediction using the resulting model
preds = model.predict(test_pool)

In [ ]:
model.get_best_score()

In [ ]:
X_test.head()

In [ ]:
y_test.head()

In [ ]:
model.predict(X_test.iloc[1,:])

In [ ]:
y_test.iloc[1,:]

In [ ]:
model.predict(X_test.iloc[1,:])-y_test.iloc[1,:]

In [ ]:
pred = model.predict(test)

In [ ]:
pred[1,:] # will output the target for the features in 2nd row of the test set given

Since there are no labels in test set we could not verify the accuacy. 

In [ ]:
y_test.head()

In [ ]:
sns.scatterplot(x=preds[:,0],y=y_test.iloc[:,0]) #target_carbon_monoxide

Seems good with carbon monoxide predictions

In [ ]:
sns.scatterplot(x=preds[:,1],y=y_test.iloc[:,1]) #target_benzene

Pretty good with benzene too.

In [ ]:
sns.scatterplot(x=preds[:,2],y=y_test.iloc[:,2]) #target_nitrogen_oxides

Here things seem to get messy. Still does not seem to be too bad.

Thank you very much for reading. I appreciate the comment or suggestion.

In [ ]:
pred.shape

In [ ]:
y_test.columns

In [ ]:
pd.DataFrame(pred, columns=y_test.columns).head() # Predictions of the test csv

In [ ]:
test.columns

In [ ]:
test.index

In [ ]:
output = pd.DataFrame(pred, columns=y_test.columns, index= test.index)
output.to_csv('Output.csv',index=True)


In [ ]:
pd.read_csv('./Output.csv').head()
